# Modifying Book Data

I wrote a book, but I want to attribute it to someone else.

In [1]:
import json
from concurrent.futures import ProcessPoolExecutor, as_completed
from os import environ
from pathlib import Path
from typing import Any, Collection, Dict

import mutf8
import pandas as pd
from IPython.display import Markdown, display
from nbt import nbt, region

In [2]:
save_folder = Path(environ["SAVE_PATH"])

In [3]:
def summarize_keystore(keystore: Dict[str, Any]) -> None:
    """Display a summary of the contents of a key-value store

    Parameters
    ----------
    keystore : dict
        The keystore to summarize

    Returns
    -------
    None
    """

    def _summarize_keystore(keystore: Dict[str, Any]) -> str:
        summary = ""
        for k, v in keystore.items():
            summary += f"\n - `{k}` : "
            if isinstance(v, (str, nbt.TAG_String)):
                summary += f'`"{v}"`'
            elif not isinstance(v, Collection):
                summary += f"`{str(v)}`"
            else:
                length = len(v)
                if 0 < length < 3:
                    summary += "\n"
                    if not isinstance(v, Dict):
                        v = {i: item for i, item in enumerate(v)}
                    summary += "\n".join(
                        (f"\t{line}" for line in _summarize_keystore(v).split("\n"))
                    )
                else:
                    summary += f"({len(v)} items)"
        return summary

    display(Markdown(_summarize_keystore(keystore)))

## Get My Inventory

Start by loading my player data

In [4]:
my_data = nbt.NBTFile(
    save_folder / "playerdata" / "8bcf4ff6-07ab-4cdb-98d0-8fcfee524197.dat"
)
summarize_keystore(my_data)


 - `foodTickTimer` : `0`
 - `AbsorptionAmount` : `0.0`
 - `XpTotal` : `45566`
 - `SelectedItemSlot` : `0`
 - `playerGameType` : `1`
 - `SpawnX` : `-1402`
 - `Invulnerable` : `0`
 - `SpawnAngle` : `-111.461669921875`
 - `SpawnDimension` : `"minecraft:overworld"`
 - `Score` : `45566`
 - `attributes` : (21 items)
 - `Tags` : 
	
	 - `0` : `"end_gravity"`
	 - `1` : `"ch_east"`
 - `UUID` : (4 items)
 - `XpLevel` : `64`
 - `Inventory` : (35 items)
 - `XpP` : `0.05496596544981003`
 - `foodLevel` : `20`
 - `LastDeathLocation` : 
	
	 - `0` : `"pos"`
	 - `1` : `"dimension"`
 - `EnderItems` : (27 items)
 - `Motion` : (3 items)
 - `Fire` : `-20`
 - `foodExhaustionLevel` : `3.3714730739593506`
 - `seenCredits` : `1`
 - `DeathTime` : `0`
 - `recipeBook` : (10 items)
 - `OnGround` : `1`
 - `spawn_extra_particles_on_fall` : `0`
 - `Brain` : 
	
	 - `0` : `"memories"`
 - `Dimension` : `"minecraft:the_end"`
 - `abilities` : (7 items)
 - `Rotation` : 
	
	 - `0` : `6.558692455291748`
	 - `1` : `34.042808532714844`
 - `HurtByTimestamp` : `88194`
 - `foodSaturationLevel` : `13.40000057220459`
 - `XpSeed` : `-367039716`
 - `Air` : `300`
 - `SpawnY` : `80`
 - `SpawnZ` : `-3983`
 - `DataVersion` : `4189`
 - `FallDistance` : `0.0`
 - `SleepTimer` : `0`
 - `FallFlying` : `0`
 - `warden_spawn_tracker` : (3 items)
 - `HurtTime` : `0`
 - `SpawnForced` : `0`
 - `Health` : `20.0`
 - `Pos` : (3 items)
 - `current_impulse_context_reset_grace_time` : `0`
 - `PortalCooldown` : `0`
 - `ignore_fall_damage_from_current_explosion` : `0`

In [5]:
summarize_keystore(my_data["Inventory"][0])


 - `count` : `1`
 - `Slot` : `0`
 - `components` : 
	
	 - `0` : `"minecraft:written_book_content"`
	 - `1` : `"minecraft:custom_model_data"`
 - `id` : `"minecraft:written_book"`

In [6]:
summarize_keystore(
    my_data["Inventory"][0]["components"]["minecraft:written_book_content"]
)


 - `pages` : (8 items)
 - `author` : `"OpenBagTwo"`
 - `title` : 
	
	 - `0` : `"raw"`
 - `resolved` : `1`

In [7]:
my_data["Inventory"][0]["components"]["minecraft:written_book_content"][
    "author"
].value = "GumboTrafe"

In [8]:
my_data["Inventory"][0]["components"]["minecraft:written_book_content"]["author"].value

'GumboTrafe'

In [9]:
my_data.write_file(
    save_folder / "playerdata" / "8bcf4ff6-07ab-4cdb-98d0-8fcfee524197.dat"
)

That was easy!

### Let's do the backup file too

Just to be sure

In [10]:
my_data = nbt.NBTFile(
    save_folder / "playerdata" / "8bcf4ff6-07ab-4cdb-98d0-8fcfee524197.dat"
)
my_data["Inventory"][0]["components"]["minecraft:written_book_content"][
    "author"
].value = "GumboTrafe"
my_data.write_file(
    save_folder / "playerdata" / "8bcf4ff6-07ab-4cdb-98d0-8fcfee524197.dat_old"
)

## That didn't work!

No idea why. But in the end I did it with commands:

1. First I dropped the book
1. Then I ran the command:
   ```mcfunction
   /data get entity @n[type=item] Item.components."minecraft:written_book_content".author
   ```
   to ensure I had the right target
1. Then I modified the attribute via:
    ```mcfunction
   /data modify entity @n[type=item] Item.components."minecraft:written_book_content".author set value "Capt. Gumbo Trafe"
   ```
1. And finally I picked up the book and verified the data change